# Demo of Synapse API

## Log onto Synapse

This is the code used to Log onto Synapse

In [1]:
import os
import synapseclient
import yaml

syn = synapseclient.Synapse()

config = yaml.load(open(os.path.join(os.path.expanduser("~"),".server_config/synapse.cfg")))

syn.login(config.get('user'), config.get('password'))

Welcome, abonil91!


## Uploading Data to the NCANDA Project

This notebook demonstrates how to upload files to the NCANDA Project using the Synapse API for Python.

First, the necessary libaries are importated.

In [2]:
from synapseclient import Project, Folder, File

The Code to Create a Project:

    project = Project('My uniquely named project')
    project = syn.store(project)

The Code to Create a File:

    data_folder = Folder('Data', parent=project)
    data_folder = syn.store(data_folder)

The Code to Add File:

       test_entity = File('/path/to/data/file.xyz', description='Fancy new data', parent=data_folder)
       test_entity = syn.store(test_entity)

### Example Upload 

In [20]:
###Select Project###
project = syn.get('syn3565171')

###Creating Folder in Project###
data_folder = Folder('Example_Upload', parent=project)
data_folder = syn.store(data_folder)

###Uploading File###
example = File('./examplefile.csv', description = "Example CSV File Upload", parent = data_folder)
example = syn.store(example)


##################################################
 Uploading file to Synapse storage 
##################################################
Uploaded Chunks [####################]100.00%     23.0bytes/23.0bytes ./examplefile.csv Done...
Upload completed in 3 seconds.


## Permissions and Controlling Access 

By default, data sets in Synapse are private to the user account.

The following function allows one to view the permissions a specific user has for a specific data set:

    Synapse.getPermissions(entity, principalId=None)

The following function allows one to set the permissions for a specific user in regards to a specific data set:

    Synapse.setPermissions(entity, principalId=None, accessType=[u'READ'], modify_benefactor=False, warn_if_inherits=True, overwrite=True)
    
Parameters:	

* entity – An Entity or Synapse ID to lookup or modify
* principalId – Identifier of a user or group
* accessType – Type of permission to be granted
* modify_benefactor – Set as True when modifying a benefactor’s ACL
* warn_if_inherits – Set as False, when creating a new ACL. Trying to modify the ACL of an Entity that inherits its ACL will result in a warning
* overwrite – By default this function overwrites existing permissions for the specified user. Set this flag to False to add new permissions nondestructively.
